In [29]:
# 03_log_creation
# Transforms data into an event log
# Events
# ID_NOTICE_CN -> case-id
# DT_DISPATCH (date) -> PUBLICATION
# DT_APPLICATIONS (date) -> PARTICIPATION
# DT_AWARD (date) -> AWARD
# CONTRACT_START (date) -> CONTRACT-START
# CONTRACT_COMPLETION (date) -> CONTRACT-END

In [30]:
# Force to reload extrernal modules every new cell execution
%reload_ext autoreload
%autoreload 2

In [31]:
### IMPORT ###
from pathlib import Path
import csv
from datetime import datetime
import pandas as pd

In [32]:
### LOCAL IMPORT ###
from config import config_reader
from utilities import read_csv_data, fix_date, json_data_to_list_dict, dic_get_years

In [60]:
### GLOBALS ###
yaml_config = config_reader.config_read_yaml("config.yml", "config")
# print(yaml_config) # debug
data_dir = str(yaml_config["DATA_DIR"])
log_dir = str(yaml_config["LOG_DIR"])
stats_dir = str(yaml_config["STATS_DIR"])
ted_cfc_file = str(yaml_config["TED_CFC_FILE"]) # input
ted_can_file = str(yaml_config["TED_CAN_FILE"]) # input
ted_config_file = str(yaml_config["TED_CONFIG_FILE"]) # input: filter configuration
dic_types_cfc = dict(yaml_config["TED_CFC_TYPES"]) # input
dic_types_can = dict(yaml_config["TED_CAN_TYPES"]) # input
list_ted_log_events = list(yaml_config["TED_LOG_EVENTS"]) # input
list_ted_log_attributes = list(yaml_config["TED_LOG_ATTRIBUTES"])
ted_join_file = str(yaml_config["TED_JOIN_FILE"]) # output
log_file = str(yaml_config["LOG_FILE"]) # output

In [34]:
### MAIN ###
print()
print("*** PROGRAM START ***")
print()

start_time = datetime.now().replace(microsecond=0)
print("Start process: ", str(start_time))
print()


*** PROGRAM START ***

Start process:  2024-09-07 15:48:03



In [35]:
# Gets filters from JSON configuration
print(">> Filters configuration")
print("Configuration file:", ted_config_file)
list_filters = json_data_to_list_dict(ted_config_file)
print("Configuration list:", list_filters) 
# Find the dictionary with the key 'YEAR' and get min/max values
min_year, max_year = dic_get_years(list_filters, 'YEAR')

>> Filters configuration
Configuration file: ted_config_data.json
Configuration list: [{'CAE_TYPE': ['3']}, {'ISO_COUNTRY_CODE': ['IT', 'FR', 'ES', 'DE', 'PT']}, {'YEAR': [2016, 2017, 2018, 2019, 2020, 2021, 2022]}]
Minimum value for YEAR: 2016
Maximum value for YEAR: 2022


In [36]:
# Reads CFC and CAN type CVS
print(">> Reading CFC and CAN files")
path_cfc_file = Path(data_dir) / ted_cfc_file.replace("YS", str(min_year)).replace("YE", str(max_year))
print("Reading:", str(path_cfc_file))
df_cfc = read_csv_data(path_cfc_file, dic_types_cfc, ";")
df_cfc_len = len(df_cfc)
print("Dataframe CFC length:", df_cfc_len)
path_can_file = Path(data_dir) / ted_can_file.replace("YS", str(min_year)).replace("YE", str(max_year))
print("Reading:", str(path_can_file))
df_can = read_csv_data(path_can_file, dic_types_can, ";")
df_can_len = len(df_can)
print("Dataframe CAN length:", df_can_len)
print()

>> Reading CFC and CAN files
Reading: data/TED_CFC_2016-2022.csv
Dataframe CFC length: 821187
Reading: data/TED_CAN_2016-2022.csv
Dataframe CAN length: 738302



In [37]:
# Join the CFC and CAN dataframe
# The two datasets are linked by CFC.FUTURE_CAN_ID = CAN.ID_NOTICE_CAN. Note: the CAN of a CFC may be missing, due to the tender (CFC.ID_NOTICE_CN) not having been awarded.
print(">> Joining CFC and CAN dataframes")

path_out = Path(data_dir) / ted_join_file.replace("YS", str(min_year)).replace("YE", str(max_year))

df_join = None

if path_out.exists():
    print("The CFC and CAN merged file already exists:", path_out)
    print(f"If you wish to recreate it, delete the file '{path_out}'")
else:
    # Join and clean
    df_join = pd.merge(left = df_cfc, right = df_can, left_on='FUTURE_CAN_ID', right_on='ID_NOTICE_CAN', how = 'inner')

    # Drop columns with '_y'
    cols_to_drop = [col for col in df_join.columns if col.endswith('_y')]
    df_join.drop(columns=cols_to_drop, inplace=True)
    # Rename columns with '_x'
    rename_dict = {col: col.rstrip('_x') for col in df_join.columns if col.endswith('_x')}
    df_join.rename(columns=rename_dict, inplace=True)

    # Output and Save
    print("Joint dataframe shape:", df_join.shape)
    df_join.to_csv(path_out, sep=";", index=False, quoting=csv.QUOTE_NONNUMERIC)
    print("Data saved to:", str(path_out)) 
print()

>> Joining CFC and CAN dataframes
Joint dataframe shape: (8608331, 96)
Data saved to: data/TED_CFC_CAN_2016-2022.csv



In [38]:
# Preparing the output
print(">> Preparing the output")
out_dir = Path(log_dir)
print("Event log directory:", str(out_dir))
out_dir.mkdir(exist_ok=True)
print()

>> Preparing the output
Event log directory: data_log



In [39]:
# List of features of interest for the event log
print(">> Creating the event log")
print("Features used as trace event:", list_ted_log_events)
print("Features used as trace attribute:", list_ted_log_attributes)
list_ted_log_features = list_ted_log_events + list_ted_log_attributes
print("Features (all):", list_ted_log_features)
print()

>> Creating the event log
Features used as trace event: ['ID_NOTICE_CN', 'DT_DISPATCH', 'CONTRACT_START', 'CONTRACT_COMPLETION', 'DT_APPLICATIONS', 'DT_AWARD']
Features used as trace attribute: ['TYPE_OF_CONTRACT', 'VALUE_EURO', 'B_ELECTRONIC_AUCTION', 'B_FRA_AGREEMENT', 'TAL_LOCATION_NUTS', 'CPV', 'ISO_COUNTRY_CODE', 'YEAR']
Features (all): ['ID_NOTICE_CN', 'DT_DISPATCH', 'CONTRACT_START', 'CONTRACT_COMPLETION', 'DT_APPLICATIONS', 'DT_AWARD', 'TYPE_OF_CONTRACT', 'VALUE_EURO', 'B_ELECTRONIC_AUCTION', 'B_FRA_AGREEMENT', 'TAL_LOCATION_NUTS', 'CPV', 'ISO_COUNTRY_CODE', 'YEAR']



In [40]:
# Select from the full dataframe only the columns in the feature list
print("> Starting dataframe")
df_ted_log = df_join[list_ted_log_features]
print("Columns considered:", df_ted_log.columns)

> Starting dataframe
Columns considered: Index(['ID_NOTICE_CN', 'DT_DISPATCH', 'CONTRACT_START', 'CONTRACT_COMPLETION',
       'DT_APPLICATIONS', 'DT_AWARD', 'TYPE_OF_CONTRACT', 'VALUE_EURO',
       'B_ELECTRONIC_AUCTION', 'B_FRA_AGREEMENT', 'TAL_LOCATION_NUTS', 'CPV',
       'ISO_COUNTRY_CODE', 'YEAR'],
      dtype='object')


In [41]:
# Checking distinct feature values
print("> Checking distinct feature values")
print("B_ELECTRONIC_AUCTION:", df_ted_log["B_ELECTRONIC_AUCTION"].unique())
print("B_FRA_AGREEMENT:", df_ted_log["B_FRA_AGREEMENT"].unique())
print("TYPE_OF_CONTRACT:", df_ted_log["TYPE_OF_CONTRACT"].unique())
print("ISO_COUNTRY_CODE:", df_ted_log["ISO_COUNTRY_CODE"].unique())
print("YEAR:", df_ted_log["YEAR"].unique())
print()

> Checking distinct feature values
B_ELECTRONIC_AUCTION: ['N' nan 'Y']
B_FRA_AGREEMENT: ['N' 'Y']
TYPE_OF_CONTRACT: ['U' 'S' 'W']
ISO_COUNTRY_CODE: ['ES' 'DE' 'FR' 'IT' 'PT']
YEAR: [2016 2017 2018 2019 2020 2021 2022]



In [42]:
print("> Creating the event log")

# Removes rows with ID_NOTICE_CN null 
df_country = df_ted_log[df_ted_log['ID_NOTICE_CN'].notna()]

# Event log header
event_log = pd.DataFrame(columns=['case_id', 'event', 'timestamp', 't_type', 'amount', 'electronic', 'framework_agr', 'nuts', 'country', 'cpv_division', 'cpv'], dtype=object) # All columns have the default type 'object'.
event_log['amount'] = event_log['amount'].astype(float)

# print("Event log header:", event_log.columns) # debug

# Events (condition T/F, event name, column feature); the column feature contains the timestamp
# (ID_NOTICE_CN is the case-id, the other columns contain the timestamp so if they are not empty the event has occurred)
conditions = [
    (~df_country['DT_DISPATCH'].isna(), 'PUBLICATION', 'DT_DISPATCH'),
    (~df_country['DT_APPLICATIONS'].isna(), 'PARTICIPATION', 'DT_APPLICATIONS'),
    (~df_country['DT_AWARD'].isna(), 'AWARD', 'DT_AWARD'),
    (~df_country['CONTRACT_START'].isna(), 'CONTRACT-START', 'CONTRACT_START'),
    (~df_country['CONTRACT_COMPLETION'].isna(), 'CONTRACT-END', 'CONTRACT_COMPLETION'),
]

for condition, event_name, date_col in conditions:
    temp_df = df_country[condition].copy()  # .copy() to avoid SettingWithCopyWarning
    temp_df['event'] = event_name
    temp_df['timestamp'] = temp_df[date_col].apply(fix_date)
    temp_df['case_id'] = temp_df['ID_NOTICE_CN']
    temp_df['t_type'] = temp_df['TYPE_OF_CONTRACT']
    temp_df['amount'] = temp_df['VALUE_EURO'].astype(float)
    temp_df['electronic'] = temp_df['B_ELECTRONIC_AUCTION']
    temp_df['framework_agr'] = temp_df['B_FRA_AGREEMENT']
    temp_df['nuts'] = temp_df['TAL_LOCATION_NUTS']
    temp_df['country'] =  temp_df['ISO_COUNTRY_CODE']
    temp_df['cpv_division'] = temp_df['CPV'].str[:2]
    temp_df['cpv'] = temp_df['CPV']    
    
    # Let us ensure that all necessary columns are present before concatenating
    temp_df = temp_df[['case_id', 'event', 'timestamp', 't_type', 'amount', 'electronic', 'framework_agr', 'nuts', 'country', 'cpv_division', 'cpv']] # same as event log header
    event_log = pd.concat([event_log, temp_df])

event_log = event_log.drop_duplicates()
print("[OK]")

print("> Ordering the event log")
# Order by 'case_id' and 'timestamp'
event_log = event_log.sort_values(by=['case_id', 'timestamp'])
print("[OK]")

print("> Adding case frequencies")
# Order by 'case_id' and 'timestamp'
event_log['case_len'] = event_log.groupby('case_id')['case_id'].transform('count')
print("[OK]")

print()
print("Cases in the final event log:", event_log['case_id'].nunique())

> Creating the event log
[OK]
> Ordering the event log
[OK]
> Adding case frequencies
[OK]

Cases in the final event log: 229632


In [43]:
print("> Adding case frequencies stats")
# Creating the dataframe with the 'case_len' frequency
df_unique = event_log.drop_duplicates(subset=['case_id']) # get one row per case
case_len_freq = df_unique['case_len'].value_counts().reset_index()
case_len_freq.columns = ['case_len', 'count']
# Calculate the percentage of occurrences
case_len_freq['count_perc'] = round((case_len_freq['count'] / case_len_freq['count'].sum()),2) * 100
# Sort the dataframe by 'case_len'
case_len_freq = case_len_freq.sort_values(by='count', ascending=False).reset_index(drop=True)
case_len_freq

> Adding case frequencies stats


,case_len,count,count_perc
0,3,101472,44.0
1,5,86125,38.0
2,4,21443,9.0
3,2,11798,5.0
4,6,4530,2.0
...,...,...,...
78,43,1,0.0
79,241,1,0.0
80,210,1,0.0
81,80,1,0.0


In [44]:
print("> Case length frequencies stats saving")
name = f"TED_CFC_CAN_{str(min_year)}-{str(max_year)}_stats"
log_stats_csv = f"{name}.csv"
log_stats_xls = f"{name}.xlsx"
path_out = Path(log_dir) / log_stats_csv
case_len_freq.to_csv(path_out, index=False, sep=';')
path_out = Path(log_dir) / log_stats_xls
case_len_freq.to_excel(path_out, index=False, sheet_name=name)

> Case length frequencies stats saving


In [45]:
print("> Saving the final event log")
log_file = log_file.replace("YS", str(min_year)).replace("YE", str(max_year))
path_out = Path(log_dir) / log_file
print("Saving event log to:", str(path_out))
event_log.to_csv(path_out, index=False, sep=';')
print("[OK]")
print("Total number of cases:", event_log['case_id'].nunique())

> Saving the final event log
Saving event log to: data_log/TED_log_2016-2022.csv
[OK]
Total number of cases: 229632


In [46]:
print(">> Splitting the event log by country")
list_countries = list(event_log["country"].unique()) # get the coutries
print("Available countries:",list_countries)
for country in list_countries:
    df_country = event_log[event_log["country"]==country] # dataframe by country
    log_file_name = f"{log_file.split('.')[0]}_{country}.csv"
    path_out = Path(log_dir) / log_file_name
    print(f"Saving event log of country '{country}' to:", str(path_out))
    df_country.to_csv(path_out, index=False, sep=';')
    print(f"Total number of cases for country '{country}': {df_country['case_id'].nunique()}")

>> Splitting the event log by country
Available countries: ['ES', 'DE', 'FR', 'IT', 'PT']
Saving event log of country 'ES' to: data_log/TED_log_2016-2022_ES.csv
Total number of cases for country 'ES': 33033
Saving event log of country 'DE' to: data_log/TED_log_2016-2022_DE.csv
Total number of cases for country 'DE': 111023
Saving event log of country 'FR' to: data_log/TED_log_2016-2022_FR.csv
Total number of cases for country 'FR': 66382
Saving event log of country 'IT' to: data_log/TED_log_2016-2022_IT.csv
Total number of cases for country 'IT': 16861
Saving event log of country 'PT' to: data_log/TED_log_2016-2022_PT.csv
Total number of cases for country 'PT': 2333


In [52]:
# Filter about end-point
print(">> Filtering end-point")

start_event = "PUBLICATION" # <-- INPUT
end_event = "CONTRACT-END" # <-- INPUT

print("Start event:", start_event)
print("End event:", end_event)

df = event_log.copy()
df = df.sort_values(by=['case_id', 'timestamp'])

# Group by case_id to get the first and last event for each case
first_event = df.groupby('case_id').first().reset_index()
last_event = df.groupby('case_id').last().reset_index()

# Filter case_ids where the first event is 'PUBLICATION' and the last event is 'CONTRACT-END'
filtered_cases = first_event[(first_event['event'] == start_event) & (last_event['event'] == end_event)]

# Get the case_ids that meet the filter criteria
filtered_case_ids = filtered_cases['case_id']

# Filter the original DataFrame to include only the filtered case_ids
df_filtered = df[df['case_id'].isin(filtered_case_ids)]

print("Filtered DF by end-point distint cases:", df_filtered["case_id"].nunique())

>> Filtering end-point
Start event: PUBLICATION
End event: CONTRACT-END
Filtered DF by end-point distint cases: 99637


In [65]:
# Stats about case duration by country
print(">> Stats about case duration by country")
# Now calculate the duration for each filtered case_id
case_durations = df_filtered.groupby('case_id').agg(
    start_time=('timestamp', 'first'),
    end_time=('timestamp', 'last'),
    country=('country', 'first')  # Assume the country is the same for the entire case_id
)

# Convert 'start_time' and 'end_time' to datetime, just in case they are still strings
case_durations['start_time'] = pd.to_datetime(case_durations['start_time'])
case_durations['end_time'] = pd.to_datetime(case_durations['end_time'])

# Calculate the duration in seconds
case_durations['duration_in_seconds'] = (case_durations['end_time'] - case_durations['start_time']).dt.total_seconds()

# Convert the duration into months (approximate)
case_durations['duration_months'] = case_durations['duration_in_seconds'] / (30.44 * 24 * 3600)  # Approximate months

# Group by 'country' and calculate the mean, median, and standard deviation for durations in months
stats_per_country = case_durations.groupby('country').agg(
    num_case_ids=('country', 'size'),  # Number of distinct case_ids for each country
    mean_duration_months=('duration_months', 'mean'),
    median_duration_months=('duration_months', 'median'),
    std_duration_months=('duration_months', 'std')
).reset_index()

# Round the results to 2 decimal places
stats_per_country = stats_per_country.round({
    'mean_duration_months': 2,
    'median_duration_months': 2,
    'std_duration_months': 2
})

>> Stats about case duration by country


In [66]:
stats_per_country

,country,num_case_ids,mean_duration_months,median_duration_months,std_duration_months
0,DE,86674,25.17,17.35,23.02
1,ES,1474,24.29,23.05,15.91
2,FR,9228,36.99,27.69,28.22
3,IT,2220,41.70,38.58,24.05
4,PT,41,19.77,14.62,11.82


In [67]:
print(">> Saving stats per country")
file_stats = Path(stats_dir) / "event_log_country_duration_stats.csv"
stats_per_country.to_csv(file_stats, sep=";", index=False)

>> Saving stats per country


In [49]:
end_time = datetime.now().replace(microsecond=0)
delta_time = end_time - start_time

print()
print("End process:", end_time)
print("Time to finish:", delta_time)
print()

print()
print("*** PROGRAM END ***")
print()


End process: 2024-09-07 15:51:41
Time to finish: 0:03:38


*** PROGRAM END ***

